# Energy Loss Scoring


## Import Libraries

In [ ]:
import os
import requests
import json
import ast
import pandas as pd
import yaml

## Retrieve the Credential

In [ ]:
credentials_file = '../../config/credentials/wml-credentials.json'

with open(credentials_file, "r") as f:
    credentials = json.load(f)

In [ ]:
# Assuming you have these variables set somewhere in your script
USERNAME = credentials["username"]
API_KEY = credentials["apikey"]
DATAPLATFORM_URL = credentials["url"]

In [ ]:
headers = {
    'Content-type': 'application/json'
}
data = {
    'username': USERNAME,
    'api_key': API_KEY
}

In [ ]:
response = requests.post(
    f"{DATAPLATFORM_URL}/icp4d-api/v1/authorize", 
    headers=headers, 
    data=json.dumps(data),
    verify=False  # This corresponds to the '-k' or '--insecure' flag in curl
)
token = response.text
dict_token = ast.literal_eval(token)
bearer_token = dict_token['token']

## Load Dataset

In [ ]:
df = pd.read_csv('../data/training.csv')

In [ ]:
df["Timestamp"] = df["Timestamp"].astype(str)

#### Pick up only few lines of the rows

In [ ]:
df = df.tail(200)

In [ ]:
df.columns

In [ ]:
df_power = df[['Timestamp', 'Active Power (KW)']]

In [ ]:
df_power = df_power.reset_index(drop=True)

## Send the Data for Scoring

In [ ]:
model_factory_config_file_name = "../config/deployment_info_three_kpis_loss_rate.yml"

with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

In [ ]:
# retrieve the CP4D endpoint at the IBM Watson Studio
scoring_url = model_factory_config['cp4d_url']
scoring_url

In [ ]:
# Select the relevant columns
array_of_input_fields = ['Timestamp', 'Active Power (KW)']
array_of_values_to_be_scored = df_power[array_of_input_fields].values.tolist()

print("Input Data")
print(array_of_input_fields)
df[array_of_input_fields].head(10)

In [ ]:
array_of_values_to_be_scored = df_power[array_of_input_fields].values.tolist()

In [ ]:
%%time
import urllib3, requests, json

# we can have this from 
#scoring_url = 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/60a32c38-e993-4f87-91c2-519b3df7b902/predictions?version=2023-11-09'
#scoring_url = 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/d5744d1b-e921-462c-bfe4-d77ca2489968/predictions?version=2023-11-16'

# NOTE: you must construct mltoken based on provided documentation
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + bearer_token}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": array_of_input_fields, "values": array_of_values_to_be_scored}]}
print("Input payload")
print(json.dumps(payload_scoring, indent=2))

response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header, verify=False)
print("Scoring response")
predictions = json.loads(response_scoring.text)
print(json.dumps(predictions, indent=2))

In [ ]:
output = predictions["predictions"][0]["values"]

In [ ]:
output

## Combine the Results

In [ ]:
reshaped_data = [row[0] for row in output]

In [ ]:
column_names = ['load_factor','loss_factor','energyloss']

In [ ]:
predictions_df = pd.DataFrame(reshaped_data, columns=column_names)

In [ ]:
predictions_df = predictions_df.astype(int)

In [ ]:
display(predictions_df)

In [ ]:
df_merged = pd.concat([df_power, predictions_df], axis=1)

In [ ]:
display(df_merged.tail(10))